In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import re

In [4]:
r = requests.get('https://go.drugbank.com/drugs?utf8=%E2%9C%93&approved=0&approved=1&nutraceutical=0&nutraceutical=1&illicit=0&illicit=1&investigational=0&investigational=1&withdrawn=0&withdrawn=1&experimental=0&experimental=1&us=0&ca=0&eu=0&commit=Apply+Filter')
print(r.status_code)
html = r.content
soup = BeautifulSoup(html, 'html.parser')

200


All the filters are selected on the cell here above to collect the largest dataset

In [5]:
links = soup.find_all('a')
links_db = re.findall('/drugs/DB\d+', str(links))
links_db = pd.DataFrame(links_db, columns = ['DB_numbers'])
len(links_db)

25

As expected, we get 25 results per page

In [6]:
df_db_num = links_db.copy()
df_db_num['DB_numbers'] = links_db['DB_numbers'].str.replace('/drugs/','')
#Idk if i'm gonna use it, but I'd like to keep a list with DB codes aswell

In [7]:
total_results = int(soup.find('div', attrs = {'class':'page_info'}).get_text().split()[-3])
total_results

11805

In [8]:
results_pages = list(range(474))
results_pages = results_pages[1:]
print(f'First page: {results_pages[0]}, last page: {results_pages[-1]}')

First page: 1, last page: 473


### **CAREFUL: THE CODE IN THE CELL HEREUNDER TAKES 4-6 MIN TO RUN**

In [9]:
substances = []

for i in results_pages:
    try:
        r = requests.get(f'https://go.drugbank.com/drugs?approved=1&c=name&ca=0&d=up&eu=0&experimental=1&illicit=1&investigational=1&nutraceutical=1&page={i}&us=0&withdrawn=1')
        soup = BeautifulSoup(r.content, 'html.parser')
        substances += soup.find_all('td', attrs={'class':'name-value text-sm-center drug-name'})
        page_suffix = [substance.find('a').get('href') for substance in substances]
        print(i)
    except:
        print(f'Error on page {i}.')
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
len(substances)

11805

In [11]:
page_suffix = [substance.find('a').get('href') for substance in substances]
len(page_suffix)

11805

### **CAREFUL: THE CELL BELOW TAKES 1 HOUR TO RUN. IF IT HAS TE BE MODIFIED FOR A TEST, ONLY RUN ON 500-1000 FIRST RESULTS**

In [13]:
gen_name = []
DB_acc_num =[]
groups = []
mol_weight = []
formula = []
inchi = []
IUPAC_name = []
smiles = []
#properties_pred = [] #LIST NOT USED AT THE END
water_solubility_pred = []
log_p_pred = []
log_s_pred = []
phys_charge = []
h_acceptor_count = []
h_donor_count = []
polar_surface_area = []
rot_bond_count = []
polarizability =[]
rule_five = []
#log_p_exp = [] #LIST NOT USED AT THE END
#log_s_exp = [] #LIST NOT USED AT THE END
#pka_exp = [] #LIST NOT USED AT THE END

for item in page_suffix:
    r = requests.get(f'https://go.drugbank.com/{item}')
    html = r.content
    soup = BeautifulSoup(html, 'html.parser')

    try:
      gen_name.append(soup.find('dd', attrs = {'class':"col-xl-4 col-md-9 col-sm-8 pr-xl-2"}).text)
    except:
      print(f'gen_name error on page {item}')
      gen_name.append(None)
    print(len(gen_name),'/',len(page_suffix))
      
    try:
      DB_acc_num.append(soup.find_all('dd', attrs = {'class':'col-xl-4 col-md-9 col-sm-8'})[0].text)
    except:
      print(f'database accesion number error on page {item}')
      DB_acc_num.append(None)
      
    try:
      groups.append(soup.find_all('dd', attrs = {'class':'col-xl-4 col-md-9 col-sm-8'})[1].text)
    except:
      print(f'groups error on page {item}')
      groups.append(None)
      
    try:
      mol_weight.append(float(soup.find('dd', attrs = {'class':'col-xl-8 col-md-9 col-sm-8'}).text.split(' ')[1]))
    except:
      print(f'mol_weight error on page {item}')
      mol_weight.append(None)
      
    try:
      formula.append(soup.find_all('dd', attrs = {'class':'col-xl-8 col-md-9 col-sm-8'})[1].text)
    except:
      print(f'formula error on page {item}')
      formula.append(None)
      
    try:
      inchi.append(soup.find_all('div', attrs = {'class':'wrap'})[0].text)
    except:
      print(f'inchi error on page {item}')
      inchi.append(None)
      
    try:
      IUPAC_name.append(soup.find_all('div', attrs = {'class':'wrap'})[1].text)
    except:
      print(f'IUPAC_name error on page {item}')
      IUPAC_name.append(None)
      
    try:
      smiles.append(soup.find_all('div', attrs = {'class':'wrap'})[2].text)
    except:
      print(f'smiles error on page {item}')
      smiles.append(None)
      
    try: 
      water_solubility_pred.append(re.findall('\d+.\d+.mg/mL', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip(' mg/mL')) #No error raised(100 first results) 
    except:
      print(f'water_solubility error on page {item}')
      water_solubility_pred.append(None)
      
    try:
      log_p_pred.append(float(re.findall('logP*\-?\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('logP'))) #No error raised(100)
    except:
      print(f'log_p_pred error on page {item}')
      log_p_pred.append(None)
      
    try:  
      log_s_pred.append(float(re.findall('logS*\-?\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('logS'))) #No error raised(100)
    except:
      print(f'log_s_pred error on page {item}')
      log_s_pred.append(None)
        
    try:
      phys_charge.append(int(re.findall('Charge*\-?\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('Charge'))) #No error raised(100)
    except:
      print(f'phys_chage error on page {item}')
      phys_charge.append(None)
      
    try:  
      h_acceptor_count.append(int(re.findall('Acceptor.Count*\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0][-1])) #No error raised(100)
    except:
      print(f'h_acceptor_count error on page {item}')
      h_acceptor_count.append(None)
      
    try: 
      h_donor_count.append(int(re.findall('Donor.Count*\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0][-1])) #No error raised(100)
    except:
      print(f'h_donor_count error on page {item}')
      h_donor_count.append(None)
      
    try: 
      polar_surface_area.append(float(re.findall('Area*\d+.\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('Area'))) #No error raised(100)
    except:
      print(f'polar_surface_area error on page {item}')
      polar_surface_area.append(None)
      
    try:
      rot_bond_count.append(int(re.findall('Bond.Count\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('Bond Count'))) #No error raised(100)
    except:
      print(f'rot_bond_count error on page {item}')
      rot_bond_count.append(None)
      
    try:
      polarizability.append(float(re.findall('zability*\d+.\d+', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('polarizability'))) #No error raised(100)
    except:
      print(f'polarizability error on page {item}')
      polarizability.append(None)
      
    try:
      rule_five.append(re.findall('Five*\w', str(soup.find_all('table', attrs = {'id':'drug-moldb-properties'})[0].text))[0].strip('Five'))
    except:
      print(f'rule_five error on page {item}')
      rule_five.append(None)
      continue 
    


1 / 11805
2 / 11805
3 / 11805
4 / 11805
5 / 11805
6 / 11805
7 / 11805
8 / 11805
9 / 11805
10 / 11805
11 / 11805
12 / 11805
13 / 11805
14 / 11805
15 / 11805
16 / 11805
17 / 11805
18 / 11805
19 / 11805
20 / 11805
21 / 11805
22 / 11805
23 / 11805
24 / 11805
25 / 11805
26 / 11805
27 / 11805
28 / 11805
29 / 11805
30 / 11805
31 / 11805
32 / 11805
33 / 11805
34 / 11805
35 / 11805
36 / 11805
37 / 11805
38 / 11805
39 / 11805
40 / 11805
41 / 11805
42 / 11805
43 / 11805
44 / 11805
45 / 11805
46 / 11805
47 / 11805
48 / 11805
49 / 11805
50 / 11805
51 / 11805
52 / 11805
53 / 11805
54 / 11805
55 / 11805
56 / 11805
57 / 11805
58 / 11805
59 / 11805
60 / 11805
61 / 11805
62 / 11805
63 / 11805
64 / 11805
65 / 11805
66 / 11805
67 / 11805
68 / 11805
69 / 11805
70 / 11805
71 / 11805
72 / 11805
73 / 11805
74 / 11805
75 / 11805
76 / 11805
77 / 11805
78 / 11805
79 / 11805
80 / 11805
81 / 11805
82 / 11805
83 / 11805
84 / 11805
85 / 11805
86 / 11805
87 / 11805
88 / 11805
89 / 11805
90 / 11805
91 / 11805
92 / 118

548 / 11805
549 / 11805
550 / 11805
551 / 11805
552 / 11805
553 / 11805
554 / 11805
555 / 11805
556 / 11805
557 / 11805
558 / 11805
559 / 11805
560 / 11805
561 / 11805
562 / 11805
563 / 11805
564 / 11805
565 / 11805
566 / 11805
567 / 11805
568 / 11805
569 / 11805
570 / 11805
571 / 11805
572 / 11805
573 / 11805
574 / 11805
575 / 11805
576 / 11805
577 / 11805
578 / 11805
579 / 11805
580 / 11805
581 / 11805
582 / 11805
583 / 11805
584 / 11805
585 / 11805
586 / 11805
587 / 11805
water_solubility error on page /drugs/DB03690
588 / 11805
589 / 11805
590 / 11805
591 / 11805
592 / 11805
593 / 11805
594 / 11805
water_solubility error on page /drugs/DB03710
log_s_pred error on page /drugs/DB03710
595 / 11805
596 / 11805
597 / 11805
598 / 11805
599 / 11805
600 / 11805
601 / 11805
water_solubility error on page /drugs/DB06841
log_s_pred error on page /drugs/DB06841
602 / 11805
603 / 11805
604 / 11805
605 / 11805
606 / 11805
607 / 11805
608 / 11805
609 / 11805
610 / 11805
611 / 11805
612 / 11805
61

973 / 11805
974 / 11805
975 / 11805
976 / 11805
977 / 11805
978 / 11805
979 / 11805
980 / 11805
981 / 11805
982 / 11805
983 / 11805
mol_weight error on page /drugs/DB01448
formula error on page /drugs/DB01448
inchi error on page /drugs/DB01448
IUPAC_name error on page /drugs/DB01448
smiles error on page /drugs/DB01448
water_solubility error on page /drugs/DB01448
log_p_pred error on page /drugs/DB01448
log_s_pred error on page /drugs/DB01448
phys_chage error on page /drugs/DB01448
h_acceptor_count error on page /drugs/DB01448
h_donor_count error on page /drugs/DB01448
polar_surface_area error on page /drugs/DB01448
rot_bond_count error on page /drugs/DB01448
polarizability error on page /drugs/DB01448
rule_five error on page /drugs/DB01448
984 / 11805
985 / 11805
986 / 11805
987 / 11805
988 / 11805
989 / 11805
990 / 11805
991 / 11805
992 / 11805
993 / 11805
994 / 11805
995 / 11805
996 / 11805
997 / 11805
998 / 11805
999 / 11805
1000 / 11805
1001 / 11805
1002 / 11805
1003 / 11805
1004 /

1445 / 11805
1446 / 11805
1447 / 11805
1448 / 11805
1449 / 11805
1450 / 11805
1451 / 11805
1452 / 11805
1453 / 11805
water_solubility error on page /drugs/DB02681
log_s_pred error on page /drugs/DB02681
1454 / 11805
1455 / 11805
1456 / 11805
1457 / 11805
1458 / 11805
1459 / 11805
1460 / 11805
1461 / 11805
1462 / 11805
1463 / 11805
1464 / 11805
1465 / 11805
1466 / 11805
1467 / 11805
1468 / 11805
1469 / 11805
1470 / 11805
1471 / 11805
1472 / 11805
1473 / 11805
1474 / 11805
1475 / 11805
1476 / 11805
1477 / 11805
1478 / 11805
1479 / 11805
1480 / 11805
1481 / 11805
1482 / 11805
1483 / 11805
1484 / 11805
1485 / 11805
1486 / 11805
1487 / 11805
1488 / 11805
1489 / 11805
1490 / 11805
1491 / 11805
1492 / 11805
polarizability error on page /drugs/DB04086
1493 / 11805
1494 / 11805
water_solubility error on page /drugs/DB03274
log_s_pred error on page /drugs/DB03274
1495 / 11805
1496 / 11805
1497 / 11805
1498 / 11805
1499 / 11805
1500 / 11805
1501 / 11805
1502 / 11805
1503 / 11805
1504 / 11805
1505

1998 / 11805
1999 / 11805
2000 / 11805
2001 / 11805
2002 / 11805
2003 / 11805
2004 / 11805
2005 / 11805
2006 / 11805
2007 / 11805
2008 / 11805
2009 / 11805
2010 / 11805
2011 / 11805
2012 / 11805
2013 / 11805
2014 / 11805
2015 / 11805
2016 / 11805
2017 / 11805
2018 / 11805
2019 / 11805
2020 / 11805
2021 / 11805
2022 / 11805
2023 / 11805
2024 / 11805
2025 / 11805
2026 / 11805
2027 / 11805
2028 / 11805
2029 / 11805
2030 / 11805
2031 / 11805
2032 / 11805
mol_weight error on page /drugs/DB03378
formula error on page /drugs/DB03378
inchi error on page /drugs/DB03378
IUPAC_name error on page /drugs/DB03378
smiles error on page /drugs/DB03378
water_solubility error on page /drugs/DB03378
log_p_pred error on page /drugs/DB03378
log_s_pred error on page /drugs/DB03378
phys_chage error on page /drugs/DB03378
h_acceptor_count error on page /drugs/DB03378
h_donor_count error on page /drugs/DB03378
polar_surface_area error on page /drugs/DB03378
rot_bond_count error on page /drugs/DB03378
polarizabi

...

11289 / 11805
polar_surface_area error on page /drugs/DB13323
11290 / 11805
mol_weight error on page /drugs/DB11295
formula error on page /drugs/DB11295
inchi error on page /drugs/DB11295
IUPAC_name error on page /drugs/DB11295
smiles error on page /drugs/DB11295
water_solubility error on page /drugs/DB11295
log_p_pred error on page /drugs/DB11295
log_s_pred error on page /drugs/DB11295
phys_chage error on page /drugs/DB11295
h_acceptor_count error on page /drugs/DB11295
h_donor_count error on page /drugs/DB11295
polar_surface_area error on page /drugs/DB11295
rot_bond_count error on page /drugs/DB11295
polarizability error on page /drugs/DB11295
rule_five error on page /drugs/DB11295
11291 / 11805
11292 / 11805
11293 / 11805
11294 / 11805
11295 / 11805
11296 / 11805
11297 / 11805
11298 / 11805
water_solubility error on page /drugs/DB03500
11299 / 11805
11300 / 11805
11301 / 11805
water_solubility error on page /drugs/DB00505
11302 / 11805
11303 / 11805
11304 / 11805
11305 / 11805
1130

11409 / 11805
11410 / 11805
11411 / 11805
11412 / 11805
11413 / 11805
11414 / 11805
11415 / 11805
water_solubility error on page /drugs/DB03325
log_s_pred error on page /drugs/DB03325
11416 / 11805
mol_weight error on page /drugs/DB13503
formula error on page /drugs/DB13503
inchi error on page /drugs/DB13503
IUPAC_name error on page /drugs/DB13503
smiles error on page /drugs/DB13503
water_solubility error on page /drugs/DB13503
log_p_pred error on page /drugs/DB13503
log_s_pred error on page /drugs/DB13503
phys_chage error on page /drugs/DB13503
h_acceptor_count error on page /drugs/DB13503
h_donor_count error on page /drugs/DB13503
polar_surface_area error on page /drugs/DB13503
rot_bond_count error on page /drugs/DB13503
polarizability error on page /drugs/DB13503
rule_five error on page /drugs/DB13503
11417 / 11805
11418 / 11805
11419 / 11805
11420 / 11805
11421 / 11805
11422 / 11805
11423 / 11805
11424 / 11805
water_solubility error on page /drugs/DB15328
log_s_pred error on page /

11613 / 11805
11614 / 11805
11615 / 11805
11616 / 11805
mol_weight error on page /drugs/DB11094
formula error on page /drugs/DB11094
inchi error on page /drugs/DB11094
IUPAC_name error on page /drugs/DB11094
smiles error on page /drugs/DB11094
water_solubility error on page /drugs/DB11094
log_p_pred error on page /drugs/DB11094
log_s_pred error on page /drugs/DB11094
phys_chage error on page /drugs/DB11094
h_acceptor_count error on page /drugs/DB11094
h_donor_count error on page /drugs/DB11094
polar_surface_area error on page /drugs/DB11094
rot_bond_count error on page /drugs/DB11094
polarizability error on page /drugs/DB11094
rule_five error on page /drugs/DB11094
11617 / 11805
water_solubility error on page /drugs/DB00163
11618 / 11805
11619 / 11805
water_solubility error on page /drugs/DB15137
11620 / 11805
mol_weight error on page /drugs/DB05430
formula error on page /drugs/DB05430
inchi error on page /drugs/DB05430
IUPAC_name error on page /drugs/DB05430
smiles error on page /drug

11679 / 11805
water_solubility error on page /drugs/DB09461
log_s_pred error on page /drugs/DB09461
polar_surface_area error on page /drugs/DB09461
11680 / 11805
water_solubility error on page /drugs/DB09315
log_s_pred error on page /drugs/DB09315
polar_surface_area error on page /drugs/DB09315
11681 / 11805
11682 / 11805
11683 / 11805
11684 / 11805
11685 / 11805
11686 / 11805
mol_weight error on page /drugs/DB05243
formula error on page /drugs/DB05243
inchi error on page /drugs/DB05243
IUPAC_name error on page /drugs/DB05243
smiles error on page /drugs/DB05243
water_solubility error on page /drugs/DB05243
log_p_pred error on page /drugs/DB05243
log_s_pred error on page /drugs/DB05243
phys_chage error on page /drugs/DB05243
h_acceptor_count error on page /drugs/DB05243
h_donor_count error on page /drugs/DB05243
polar_surface_area error on page /drugs/DB05243
rot_bond_count error on page /drugs/DB05243
polarizability error on page /drugs/DB05243
rule_five error on page /drugs/DB05243
11

11708 / 11805
11709 / 11805
mol_weight error on page /drugs/DB05017
formula error on page /drugs/DB05017
inchi error on page /drugs/DB05017
IUPAC_name error on page /drugs/DB05017
smiles error on page /drugs/DB05017
water_solubility error on page /drugs/DB05017
log_p_pred error on page /drugs/DB05017
log_s_pred error on page /drugs/DB05017
phys_chage error on page /drugs/DB05017
h_acceptor_count error on page /drugs/DB05017
h_donor_count error on page /drugs/DB05017
polar_surface_area error on page /drugs/DB05017
rot_bond_count error on page /drugs/DB05017
polarizability error on page /drugs/DB05017
rule_five error on page /drugs/DB05017
11710 / 11805
water_solubility error on page /drugs/DB13076
log_s_pred error on page /drugs/DB13076
polar_surface_area error on page /drugs/DB13076
11711 / 11805
11712 / 11805
11713 / 11805
11714 / 11805
11715 / 11805
11716 / 11805
polarizability error on page /drugs/DB00549
11717 / 11805
11718 / 11805
11719 / 11805
11720 / 11805
11721 / 11805
11722 / 

In [14]:
print('gen_name', len(gen_name))
print('DB_acc_num', len(DB_acc_num))
print('groups', len(groups))
print('mol_weight', len(mol_weight))
print('formula', len(formula))
print('inchi', len(inchi))
print('IUPAC_name', len(IUPAC_name))
print('smiles', len(smiles))
print('water_solubility_pred', len(water_solubility_pred))
print('log_p_pred', len(log_p_pred))
print('log_s_pred', len(log_s_pred))
print('phys_charge', len(phys_charge))
print('h_acceptor_count', len(h_acceptor_count))
print('h_donor_count', len(h_donor_count))
print('polar_surface_area', len(polar_surface_area))
print('rot_bond_count', len(rot_bond_count))
print('polarizability', len(polarizability))
print('rule_five', len(rule_five))

gen_name 11805
DB_acc_num 11805
groups 11805
mol_weight 11805
formula 11805
inchi 11805
IUPAC_name 11805
smiles 11805
water_solubility_pred 11805
log_p_pred 11805
log_s_pred 11805
phys_charge 11805
h_acceptor_count 11805
h_donor_count 11805
polar_surface_area 11805
rot_bond_count 11805
polarizability 11805
rule_five 11805


In [15]:
substance_dict = { 'generic_name' : gen_name, 'DB_acc_num' : DB_acc_num, 'groups' : groups, 'mol_weight' : mol_weight, 'formula' : formula, 'inchi' : inchi, 'IUPAC_name' : IUPAC_name, 'smiles' : smiles, 'water_sol_pred_mg/mL' : water_solubility_pred, 'log_p_pred' : log_p_pred, 'log_s_pred' : log_s_pred, 'phys_charge' : phys_charge, 'h_acceptor_count' : h_acceptor_count, 'h_donor_count' : h_donor_count, 'polar_surface_area_A2' : polar_surface_area, 'rotatable_bond_count' : rot_bond_count, 'polarizability_A3' : polarizability, 'rule_five' : rule_five, }

In [16]:
df = pd.DataFrame.from_dict(substance_dict)
df.head()

,generic_name,DB_acc_num,groups,mol_weight,formula,inchi,IUPAC_name,smiles,water_sol_pred_mg/mL,log_p_pred,log_s_pred,phys_charge,h_acceptor_count,h_donor_count,polar_surface_area_A2,rotatable_bond_count,polarizability_A3,rule_five
0,(-)-menthol 1-propylene glycol carbonate,DB14136,Experimental,258.3580,C14H26O4,InChI=1S/C14H26O4/c1-9(2)12-6-5-10(3)7-13(12)1...,"1-[({[(1R,2S,5R)-5-methyl-2-(propan-2-yl)cyclo...",[H]C(C)(O)COC(=O)O[C@]1([H])C[C@]([H])(C)CC[C@...,0.194,3.0,-3.0,0.0,3.0,1.0,55.76,6.0,29.72,Y
1,"({[(3E)-2'-Oxo-2',7'-dihydro-2,3'-biindol-3(7H...",DB07949,Experimental,335.3135,C18H13N3O4,InChI=1S/C18H13N3O4/c22-14(23)9-25-21-16-11-6-...,"2-({[(3E)-2-[(3Z)-2-oxo-3,7-dihydro-2H-indol-3...",OC(=O)CO\N=C1/C2=CC=CCC2=N/C/1=C1\C(=O)N=C2CC=...,0.0442,2.0,-3.0,-1.0,7.0,1.0,100.68,3.0,34.43,Y
2,({3-[1-(4-Hydroxy-2-oxo-2H-chromen-3-yl)-propy...,DB08664,Experimental,452.4996,C25H28N2O6,InChI=1S/C25H28N2O6/c1-5-17(21-22(29)18-11-6-7...,tert-butyl N-[({3-[(1S)-1-(4-hydroxy-2-oxo-2H-...,CC[C@@H](C1=CC(NC(=O)CNC(=O)OC(C)(C)C)=CC=C1)C...,0.00464,3.0,-5.0,-1.0,4.0,3.0,113.96,8.0,48.45,Y
3,(+)-1-bromo-2-propanol,DB03335,Experimental,138.9910,C3H7BrO,"InChI=1S/C3H7BrO/c1-3(5)2-4/h3,5H,2H2,1H3/t3-/...",(2S)-1-bromopropan-2-ol,C[C@H](O)CBr,61.8,0.0,-0.0,0.0,1.0,1.0,20.23,1.0,10.23,Y
4,(+)-2-(4-biphenyl)propionic acid,DB02047,Experimental,226.2705,C15H14O2,InChI=1S/C15H14O2/c1-11(15(16)17)12-7-9-14(10-...,"(2S)-2-{[1,1'-biphenyl]-4-yl}propanoic acid",C[C@H](C(O)=O)C1=CC=C(C=C1)C1=CC=CC=C1,0.0285,3.0,-3.0,-1.0,2.0,1.0,37.30,3.0,25.22,Y


In [18]:
df.to_csv('Drugbank_complete.csv', sep = ',')

# **END**